Optimizing RFR Models based on Perovskite Compositions
======================================================

**Author:** Panayotis Manganaris



## Dependencies



In [ ]:
# featurization
import cmcl
from cmcl import Categories
# multi-criterion model evaluation
from yogi.model_selection import summarize_HPO
from yogi.model_selection import pandas_validation_curve as pvc
from yogi.metrics.pandas_scoring import PandasScoreAdaptor as PSA
from yogi.metrics.pandas_scoring import batch_score
# visualization convenience
from spyglass.model_imaging import parityplot

The Intel distribution provides accelerated ml algorithms. Run this cell before importing the algorithms. "cmcl" and "yogi" are our in-house modules for analyzing chemical compositions and enabling different nuts and bolts of ML algorithms.



In [ ]:
from sklearnex import patch_sklearn
patch_sklearn()

In [ ]:
# data tools
import sqlite3
import pandas as pd
import numpy as np
# feature engineering
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder, Normalizer, StandardScaler
# predictors
from sklearn.ensemble import RandomForestRegressor
## pipeline workflow
from sklearn.pipeline import make_pipeline as mkpipe
from sklearn.model_selection import KFold, GroupKFold
from sklearn.model_selection import learning_curve, validation_curve
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import GridSearchCV as gsCV
# model eval
from sklearn.base import clone
from sklearn.metrics import make_scorer, mean_squared_error, r2_score, explained_variance_score, max_error
import joblib
#visualization
from sklearn import set_config
import matplotlib.pyplot as plt
import seaborn as sns
# ignore all FutureWarnings -- handling coming in a future version of yogi
from warnings import simplefilter
simplefilter(action='ignore', category=FutureWarning)

## Load Data and Compute Composition Vectors



### Load Curated Subset of Mannodi Data



In [ ]:
my = pd.read_csv("./mannodi_data.csv").set_index(["index", "Formula", "sim_cell"])
lookup = pd.read_csv("./constituent_properties.csv").set_index("Formula")

### Compute Compostion Vectors using cmcl



Additionally, cmcl offers a convenience function for creating groups out of the column labels (groups = type of mixing in this dataset).



In [ ]:
mc = my.ft.comp() # compute numerical compostion vectors from strings
mc = mc.collect.abx() # convenient site groupings for perovskites data

### Generate mix category and assign to index for future imaging



Helps with visualizing the final model performance by tracking the features through all the model development transformations.



In [ ]:
mixlog = mc.groupby(level=0, axis=1).count()
mix = mixlog.pipe(Categories.logif, condition=lambda x: x>1, default="pure", catstring="and")
mc = mc.assign(mix=mix).set_index("mix", append=True)
my = my.assign(mix=mix).set_index("mix", append=True)

## Model BG Using Composition Vectors



### Make Composition Pipeline



The model we use relies on a standard chain of data preprocessing steps. These steps are encapsulated in a single meta-estimator called a "pipeline." The pipeline is instantiated with a set of default "hyperparameters."

Later, a hyperparameter optimization strategy based on exhaustive grid search will chose parameters that ensure the architecture performs as best it can for general high-accuracy prediction.



In [ ]:
fillna = SimpleImputer(strategy="constant", fill_value=0.0)
cpipe = mkpipe(fillna, Normalizer(), RandomForestRegressor())

### Scoring Scheme



The composition space is approximated by a set of discrete domains due the limited nature of the 2x2x2 supercells used to obtain this data. The pure domain is completely covered by 90 data points. The alloy domains are combinatorially large in the 14 dimensional component space under focus, and each has only been sparsely sampled.

Our primary objective is to create a surrogate model of these domains that can be used for on-demand prediction, active learning, and inverse design.

In order to target model generality, it will be necessary to score the model's performance with respect to it's predictions individually over each alloy domain and over the union of these domains.



#### Prepare subset scoring weights and ordinal group labels



In [ ]:
mixweight = pd.get_dummies(mix)
mixcat = pd.Series(OrdinalEncoder().fit_transform(mix.values.reshape(-1, 1)).reshape(-1),
                     index=mc.index).astype(int)

#### Define Scoring Metrics


Nine metrics are used to monitor the fitness of the random forest model. Some monitor the data trend and spread:

-   R<sup>2</sup>
-   Explained Variance


One monitors the largest breakdown in accuracy:

-   Max Error


The remaining six group-wise RMSE metrics monitor the accuracy for each type of data point:

-   total RMSE
-   A-site RMSE
-   B-site RMSE
-   X-site RMSE
-   XandB-site RMSE
-   pure RMSE


The PandasScoreAdaptor (PSA) ensures the prediction losses are weighted correctly when scoring as long as both the targets and the sample weights passed to the estimators are always pandas objects.



In [ ]:
site_mse = PSA(mean_squared_error).score
scorings = {'r2': make_scorer(r2_score),
            'ev': make_scorer(explained_variance_score),
            'maxerr': make_scorer(max_error, greater_is_better=False),
            'rmse': make_scorer(mean_squared_error, greater_is_better=False, squared=False),
            'A_rmse': make_scorer(site_mse, greater_is_better=False,
                                  squared=False, sample_weight=mixweight.A),
            'B_rmse': make_scorer(site_mse, greater_is_better=False,
                                  squared=False, sample_weight=mixweight.B),
            'X_rmse': make_scorer(site_mse, greater_is_better=False,
                                  squared=False, sample_weight=mixweight.X),
            'BandX_rmse': make_scorer(site_mse, greater_is_better=False,
                                      squared=False, sample_weight=mixweight.BandX),
            'Pure_rmse': make_scorer(site_mse, greater_is_better=False,
                                     squared=False, sample_weight=mixweight.pure),}

### Make Dedicated Test Train Split



A dedicated test-train split is made. This split preserves the proportion of each alloy group in the test and train partitions, which helps with the final model evaluation. 

-   All decisions about model optimization will be made using only the dedicated training partition.
-   The test partition will be reserved until a final model pipeline is parametrized and fit.
-   The predictions made on the test partition will either confirm or deny the model's ability to work outside of the training domain.



In [ ]:
sss = StratifiedShuffleSplit(n_splits=1, train_size=0.8, random_state=None)
train_idx, test_idx = next(sss.split(mc, mixcat)) #stratify split by mix categories
mc_tr, mc_ts = mc.iloc[train_idx], mc.iloc[test_idx]
my_tr, my_ts = my.iloc[train_idx], my.iloc[test_idx]
mixcat_tr, mixcat_ts = mixcat.iloc[train_idx], mixcat.iloc[test_idx]

### Learning Curves &#x2013; Using Deterministically Random Cross Validation



Cross-validation within the training set will be the only way of checking the generality of models, aside from predictions on the test set. The first step is to obtain an understanding of how much data is needed for training that can be suitably generalized.

10-fold cross-validation yields 10 sample scores at each partition size. This means 90% of the training set is used for actual training and the remaining 10% is used for validation.

The shuffling is seeded with a deterministic random state to ensure scores are comparable across partition size



In [ ]:
kf_lc = KFold(n_splits=10, shuffle=True, random_state=111)

In [ ]:
with joblib.parallel_backend('multiprocessing'):
  LC = pvc(learning_curve, cpipe, mc_tr, my_tr.PBE_bg_eV,
           train_sizes=np.linspace(0.1, 1.0, 10), cv=kf_lc, scoring=scorings)
  LC = LC.melt(id_vars=["partition"], ignore_index=False).reset_index()

Notice that the error metrics are negated so that, consistently with
the R<sup>2</sup> and ev scores, the greater the number, the better the model
performs.



In [ ]:
p = sns.FacetGrid(LC, col="score", hue="partition", col_wrap=3, sharey=False)
p.map(sns.lineplot, "train_sizes", "value")
p.add_legend()
p.figure.show()

The Random Forest's validation scores continue to rise as the partition size grows

-   Random Forest generality increases with more exposure
-   Equivalently, an insufficiently experienced random forest is biased towards what it has seen.


### Obtain Generality Baseline



The four main alloy classes are roughly equally represented in the dataset. It would be interesting to see if a model trained on only three of them performs well on the fourth.

There are four main groups, so we use four groupwise partitions. The tiny "BandX" group is mixed in with the others -- it's name is not printed by default. The splitter uses the mixcat ordinal series to ensure the index splits are correct.


In [ ]:
gkf = GroupKFold(n_splits=4)

A function is defined to streamline this test and ensure it cannot contaminate the estimator to be optimized.



In [ ]:
def test_generality(estimator, X_tr, y_tr, groups_tr, X_ts, y_ts, groups_ts):
    estimator = clone(estimator) #unfitted, cloned params
    gentpl = gkf.split(X_tr, y_tr, groups=groups_tr), gkf.split(X_ts, y_ts, groups=groups_ts)
    #train and test index generators, in order
    val_scores = []
    tst_scores = []
    for train_idx, val_idx, _, tst_idx in [sum(gengroup, ()) for gengroup in zip(*gentpl)]:
        tr_val_group_names = groups_tr.iloc[val_idx].index.get_level_values("mix").unique()
        ts_group_names = groups_ts.iloc[tst_idx].index.get_level_values("mix").unique()
        #fit to tr part
        estimator.fit(X_tr.iloc[train_idx], y_tr.iloc[train_idx])
        #get val and test scores
        tr_val_score_series = pd.Series(batch_score(estimator, X_tr.iloc[val_idx], y_tr.iloc[val_idx], **scorings))
        tr_val_score_series.name=tr_val_group_names[0]#"_&_".join(tr_val_group_names) #delete the indexed list and uncomment the join operation to see where BandX was put
        ts_score_series = pd.Series(batch_score(estimator, X_ts.iloc[tst_idx], y_ts.iloc[tst_idx], **scorings))
        ts_score_series.name=ts_group_names[0]#"_&_".join(ts_group_names)
        val_scores.append(tr_val_score_series)
        tst_scores.append(ts_score_series)
    tr_val_scores = pd.concat(val_scores, axis=1).assign(partition="validation")
    ts_scores = pd.concat(tst_scores, axis=1).assign(partition="test")
    group_scores = pd.concat([tr_val_scores, ts_scores]).round(5).drop_duplicates(keep="first")
    return group_scores

In [ ]:
test_generality(cpipe, mc_tr, my_tr.PBE_bg_eV, mixcat_tr, mc_ts, my_ts.PBE_bg_eV, mixcat_ts)

-   There are substantial errors in all the groups
    -   in the test partition, the model utterly fails in extrapolating the bandgaps of B-mixed alloys
    -   in the train partition, it's still not good
    -   however, extrapolating the other partitions is relatively much better
-   The B partition is mostly representative of the others
    -   w.r.t the underlying function as identified by this specific model architecture
    -   different architectures can be inclined to learn different characteristics of a function

**Note:** batch_score defaults to unweighted scoring if the sum of weights
in the given sample equal zero, that is why most of the group-wise rmse
scores are the same.



### Optimize Hyper-parameters for Composition Model



#### Define first level of Hyperparameter search



These cells archive the optimization process, **running them all is not necessary**. They are very time consuming. The results of the search are saved in the notebook.

**Note:** The Hyperparameter Optimization has been abridged for time. Only the largest grid-search and the final grid-search are shown. In practice, it is best to iterate through the search space as many times as are needed.

##### Construct original Hyper-parameter Space



In [ ]:
grid = [
    {'normalizer__norm': ['l1', 'l2', 'max'],
     'randomforestregressor__bootstrap': [True], #build each tree from sample
     'randomforestregressor__ccp_alpha': [0.0, 0.002], #cost-complexity pruning
     'randomforestregressor__criterion': ['squared_error', 'absolute_error', 'poisson'], #variance reductions vs deviance reduction
     #'randomforestregressor__maxBins:': [256], #this is an artifact of the intel algorithm, it is not a valid key.
     'randomforestregressor__max_depth': [25, 20], #investigate dept of constituent trees, limit
     'randomforestregressor__max_features': ['auto', 3, 5], #split after considering
     'randomforestregressor__max_leaf_nodes': [750, 800], #see depth exploration in DT notes
     'randomforestregressor__max_samples': [0.9, 0.6, 0.3], #frac to bag
     'randomforestregressor__minBinSize': [1],
     'randomforestregressor__min_impurity_decrease': [0.0, 0.3], #0.3 corresponds to the onset of aggressive ccp
     'randomforestregressor__min_samples_leaf': [1], #just sensible
     'randomforestregressor__min_samples_split': [2, 5], #
     'randomforestregressor__min_weight_fraction_leaf': [0.0], #
     'randomforestregressor__n_estimators': [20, 50, 100],
     'randomforestregressor__n_jobs': [4], #parallelize exec
     'randomforestregressor__oob_score': [True], #use out-of-bag samples to validate (faster)
     'randomforestregressor__random_state': [None], #do not touch
     'randomforestregressor__verbose': [0], 
     'randomforestregressor__warm_start': [False] #make a new forest every time (honest)
     },
    {'normalizer__norm': ['l1', 'l2', 'max'],
     'randomforestregressor__bootstrap': [False], #Build each tree from everything
     'randomforestregressor__ccp_alpha': [0.0, 0.002], #cost-complexity pruning
     'randomforestregressor__criterion': ['squared_error', 'absolute_error', 'poisson'], #variance reductions vs deviance reduction
     #'randomforestregressor__maxBins:': [256],
     'randomforestregressor__max_depth': [25, 20], #investigate dept of constituent trees, limit
     'randomforestregressor__max_features': ['auto', 3, 5], #split after considering
     'randomforestregressor__max_leaf_nodes': [750, 800], #see depth exploration in DT notes
     'randomforestregressor__max_samples': [None], #"bag" everything
     'randomforestregressor__minBinSize': [1],
     'randomforestregressor__min_impurity_decrease': [0.0, 0.3], #
     'randomforestregressor__min_samples_leaf': [1], #just sensible
     'randomforestregressor__min_samples_split': [2, 5], #
     'randomforestregressor__min_weight_fraction_leaf': [0.0], #
     'randomforestregressor__n_estimators': [20, 50, 100],
     'randomforestregressor__n_jobs': [4], #parallelize exec
     #oob score not available
     'randomforestregressor__random_state': [None], #do not touch
     'randomforestregressor__verbose': [0], 
     'randomforestregressor__warm_start': [False] #make a new forest every time (honest)
     }
]

##### Composition model of PBE_bg_eV



initially, only 3 fold validation is used to save on computation time, this is a reasonable compromise on training generality according to the learning curves.



In [ ]:
cgs = gsCV(estimator=cpipe,
            param_grid=grid,
            cv=3, verbose=1, scoring=scorings, refit="r2", return_train_score=True)

with joblib.parallel_backend('multiprocessing'):
    cgs.fit(mc_tr, my_tr.PBE_bg_eV)

Fitting 3 folds for each of 10368 candidates, totalling 31104 fits &#x2013; about 45 minutes to fit with acceleration.



##### Determine next Grid Space to explore

The cell below provides a HP search grid and applies weights of importance to each of the 9 error metrics (B_rmse is given a weight "2" to improve B-site mixed predictions further).

In [ ]:
summary, next_grid = summarize_HPO(cgs, grid, topN=10, metric_weights=[1,1,1,1,1,2,1,0,1], strategy="oavg")
summary

Mostly Equal Weights In Summary:

| |space<sub>0</sub>|space<sub>1</sub>|entropy<sub>0</sub>|entropy<sub>1</sub>|scores<sub>0</sub>|scores<sub>1</sub>|next<sub>0</sub>|next<sub>1</sub>|
|---|---|---|---|---|---|---|---|---|
| normalizer_<sub>norm</sub>|[l1, l2, max]|[l1, l2, max]|0.853155|0.853155|[19.07, 0.07, 10.74]|[19.07, 0.07, 10.74]|[l1, max]|[l1, max]|
| bootstrap|[True]|[False]|0.142457|0.289120|[26.18]|[3.69]|[True]|[False]|
| ccp<sub>alpha</sub>|[0.0, 0.002]|[0.0, 0.002]|0.485723|0.485723|[27.33, 2.54]|[27.33, 2.54]|[0.0]|[0.0]|
| criterion|[squared<sub>error</sub>, absolute<sub>error</sub>, poisson]|[squared<sub>error</sub>, absolute<sub>error</sub>, poisson]|0.687787|0.687787|[18.12, 11.75, 0.0]|[18.12, 11.75, 0.0]|[squared<sub>error</sub>, absolute<sub>error</sub>]|[squared<sub>error</sub>, absolute<sub>error</sub>]|
| max<sub>depth</sub>|[25, 20]|[25, 20]|0.683604|0.683604|[14.16, 15.72]|[14.16, 15.72]|[20]|[20]|
| max<sub>features</sub>|[auto, 3, 5]|[auto, 3, 5]|0.759206|0.759206|[25.04, 0.06, 4.78]|[25.04, 0.06, 4.78]|[auto]|[auto]|
| max<sub>leaf</sub><sub>nodes</sub>|[750, 800]|[750, 800]|0.692553|0.692553|[14.88, 15.0]|[14.88, 15.0]|[800]|[800]|
| max<sub>samples</sub>|[0.9, 0.6, 0.3]|[None]|0.812818|0.289120|[21.28, 4.82, 0.09]|[0.16]|[0.9]|[None]|
| minBinSize|[1]|[1]|-0.000000|-0.000000|NaN|NaN|[1]|[1]|
| min<sub>impurity</sub><sub>decrease</sub>|[0.0, 0.3]|[0.0, 0.3]|-0.000000|-0.000000|NaN|NaN|[0.0]|[0.0]|
| min<sub>samples</sub><sub>leaf</sub>|[1]|[1]|-0.000000|-0.000000|NaN|NaN|[1]|[1]|
| min<sub>samples</sub><sub>split</sub>|[2, 5]|[2, 5]|0.485723|0.485723|[26.1, 3.77]|[26.1, 3.77]|[2]|[2]|
| min<sub>weight</sub><sub>fraction</sub><sub>leaf</sub>|[0.0]|[0.0]|-0.000000|-0.000000|NaN|NaN|[0.0]|[0.0]|
| n<sub>estimators</sub>|[20, 50, 100]|[20, 50, 100]|1.083522|1.083522|[7.95, 8.22, 13.7]|[7.95, 8.22, 13.7]|[100]|[100]|
| n<sub>jobs</sub>|[4]|[4]|-0.000000|-0.000000|NaN|NaN|[4]|[4]|
| oob<sub>score</sub>|[True]|NaN|0.142457|NaN|[0.84]|NaN|[True]|NaN|
| random<sub>state</sub>|[None]|[None]|-0.000000|-0.000000|NaN|NaN|[None]|[None]|
| verbose|[0]|[0]|-0.000000|-0.000000|NaN|NaN|[0]|[0]|
| warm<sub>start</sub>|[False]|[False]|-0.000000|-0.000000|NaN|NaN|[False]|[False]|

-   l1 normalization is best
-   bootstrapping the regressor is much more performant
    -   90% sampling is best (rfr improves with more exposure, makes sense)
    -   notice: bootstrap sampling appears to rank only slightly more
        frequently in the top ten than no-bootstrap, but has much higher
        scores. suggesting it also dominates the highest ranks in general.
-   max normalization also does well, but not as well
-   squared error does best
-   absolute<sub>error</sub> (more expensive) is less susceptible to compromising on extremes, but appears mostly unfavorable
-   limiting tree depth slightly better than not limiting it
-   growth on all features better than growth on few features. larger axis limits yet to be explored
-   unlimited nodes marginally better than limited nodes
-   impurity decrease threshold is ineffective
-   unlimited split granularity better than limited granularity
-   generally, more estimators outperform fewer

Favorably Weighting B scores:

| |space<sub>0</sub>|space<sub>1</sub>|entropy<sub>0</sub>|entropy<sub>1</sub>|scores<sub>0</sub>|scores<sub>1</sub>|next<sub>0</sub>|next<sub>1</sub>|
|---|---|---|---|---|---|---|---|---|
| normalizer_<sub>norm</sub>|[l1, l2, max]|[l1, l2, max]|0.853155|0.853155|[21.94, 0.07, 11.25]|[21.94, 0.07, 11.25]|[l1, max]|[l1, max]|
| bootstrap|[True]|[False]|0.142457|0.289120|[29.47]|[3.79]|[True]|[False]|
| ccp<sub>alpha</sub>|[0.0, 0.002]|[0.0, 0.002]|0.485723|0.485723|[30.68, 2.58]|[30.68, 2.58]|[0.0]|[0.0]|
| criterion|[squared<sub>error</sub>, absolute<sub>error</sub>, poisson]|[squared<sub>error</sub>, absolute<sub>error</sub>, poisson]|0.687787|0.687787|[21.39, 11.87, 0.0]|[21.39, 11.87, 0.0]|[squared<sub>error</sub>, absolute<sub>error</sub>]|[squared<sub>error</sub>, absolute<sub>error</sub>]|
| max<sub>depth</sub>|[25, 20]|[25, 20]|0.683604|0.683604|[16.54, 16.72]|[16.54, 16.72]|[20]|[20]|
| max<sub>features</sub>|[auto, 3, 5]|[auto, 3, 5]|0.759206|0.759206|[27.43, 0.06, 5.77]|[27.43, 0.06, 5.77]|[auto]|[auto]|
| max<sub>leaf</sub><sub>nodes</sub>|[750, 800]|[750, 800]|0.692553|0.692553|[17.34, 15.92]|[17.34, 15.92]|[750]|[750]|
| max<sub>samples</sub>|[0.9, 0.6, 0.3]|[None]|0.812818|0.289120|[22.68, 6.69, 0.09]|[0.16]|[0.9]|[None]|
| minBinSize|[1]|[1]|-0.000000|-0.000000|NaN|NaN|[1]|[1]|
| min<sub>impurity</sub><sub>decrease</sub>|[0.0, 0.3]|[0.0, 0.3]|-0.000000|-0.000000|NaN|NaN|[0.0]|[0.0]|
| min<sub>samples</sub><sub>leaf</sub>|[1]|[1]|-0.000000|-0.000000|NaN|NaN|[1]|[1]|
| min<sub>samples</sub><sub>split</sub>|[2, 5]|[2, 5]|0.485723|0.485723|[28.39, 4.87]|[28.39, 4.87]|[2]|[2]|
| min<sub>weight</sub><sub>fraction</sub><sub>leaf</sub>|[0.0]|[0.0]|-0.000000|-0.000000|NaN|NaN|[0.0]|[0.0]|
| n<sub>estimators</sub>|[20, 50, 100]|[20, 50, 100]|1.083522|1.083522|[9.59, 8.5, 15.18]|[9.59, 8.5, 15.18]|[100]|[100]|
| n<sub>jobs</sub>|[4]|[4]|-0.000000|-0.000000|NaN|NaN|[4]|[4]|
| oob<sub>score</sub>|[True]|NaN|0.142457|NaN|[0.84]|NaN|[True]|NaN|
| random<sub>state</sub>|[None]|[None]|-0.000000|-0.000000|NaN|NaN|[None]|[None]|
| verbose|[0]|[0]|-0.000000|-0.000000|NaN|NaN|[0]|[0]|
| warm<sub>start</sub>|[False]|[False]|-0.000000|-0.000000|NaN|NaN|[False]|[False]|

-   gap between limited and unlimited tree depth closes slightly
-   limited leaf nodes becomes more favorable than unlimited leaf nodes &#x2013; reversal!
-   20 estimators actually ranks much higher. less averaging => more bias helps B



#### Skipping Iterations....

#### &#x2013; Iteratively Optimize Hyperparameters (final)



##### Construct subsequent HP space



In the limited tree domains, without interference from n_estimators, various criterion are explored in detail.



In [ ]:
grid = [
    {'normalizer__norm': ['l1'],
     'randomforestregressor__bootstrap': [True],
     'randomforestregressor__ccp_alpha': [0.0],
     'randomforestregressor__criterion': ['squared_error', 'absolute_error', 'poisson'],
     'randomforestregressor__max_depth': [15, 20],
     'randomforestregressor__max_features': ['auto'],
     'randomforestregressor__max_leaf_nodes': [600, 700, 800],
     'randomforestregressor__max_samples': [0.9],
     'randomforestregressor__minBinSize': [1],
     'randomforestregressor__min_impurity_decrease': [0.0],
     'randomforestregressor__min_samples_leaf': [1],
     'randomforestregressor__min_samples_split': [2],
     'randomforestregressor__min_weight_fraction_leaf': [0.0],
     'randomforestregressor__n_estimators': [100], #compromise in anticipation of possible overfitting
     'randomforestregressor__n_jobs': [4],
     'randomforestregressor__oob_score': [True],
     'randomforestregressor__random_state': [None],
     'randomforestregressor__verbose': [0],
     'randomforestregressor__warm_start': [False]
     }
]

##### Composition model of PBE_bg_eV



In [ ]:
cgs = gsCV(estimator=cpipe,
            param_grid=grid,
            cv=3, verbose=1, scoring=scorings, refit="r2", return_train_score=True)

with joblib.parallel_backend('multiprocessing'):
    cgs.fit(mc_tr, my_tr.PBE_bg_eV)

Fitting 3 folds for each of 18 candidates, totalling 54 fits



##### Determine next Grid Space to explore



In [ ]:
summary, next_grid = summarize_HPO(cgs, grid, topN=10, metric_weights=[1,1,1,1,1,2,1,0,1], strategy="oavg")
summary

In Summary:

| |space<sub>0</sub>|entropy<sub>0</sub>|scores<sub>0</sub>|next<sub>0</sub>|
|---|---|---|---|---|
| normalizer_<sub>norm</sub>|[l1]|-0.000000|NaN|[l1]|
| bootstrap|[True]|-0.000000|NaN|[True]|
| ccp<sub>alpha</sub>|[0.0]|-0.000000|NaN|[0.0]|
| criterion|[squared<sub>error</sub>, absolute<sub>error</sub>, poisson]|1.054920|[18.27, 14.65, 2.16]|[squared<sub>error</sub>, absolute<sub>error</sub>]|
| max<sub>depth</sub>|[15, 20]|0.673012|[10.35, 24.73]|[20]|
| max<sub>features</sub>|[auto]|-0.000000|NaN|[auto]|
| max<sub>leaf</sub><sub>nodes</sub>|[600, 700, 800]|1.098612|[8.45, 12.28, 14.35]|[700, 800]|
| max<sub>samples</sub>|[0.9]|-0.000000|NaN|[0.9]|
| minBinSize|[1]|-0.000000|NaN|[1]|
| min<sub>impurity</sub><sub>decrease</sub>|[0.0]|-0.000000|NaN|[0.0]|
| min<sub>samples</sub><sub>leaf</sub>|[1]|-0.000000|NaN|[1]|
| min<sub>samples</sub><sub>split</sub>|[2]|-0.000000|NaN|[2]|
| min<sub>weight</sub><sub>fraction</sub><sub>leaf</sub>|[0.0]|-0.000000|NaN|[0.0]|
| n<sub>estimators</sub>|[100]|-0.000000|NaN|[100]|
| n<sub>jobs</sub>|[4]|-0.000000|NaN|[4]|
| oob<sub>score</sub>|[True]|-0.000000|NaN|[True]|
| random<sub>state</sub>|[None]|-0.000000|NaN|[None]|
| verbose|[0]|-0.000000|NaN|[0]|
| warm<sub>start</sub>|[False]|-0.000000|NaN|[False]|

-   squared error and absolute error perform similarly in this area
    -   absolute error is more expensive to use,
    -   choosing square error is justifiable for practicality
-   gentle limits are preferred to hard limits

B score weighting makes no difference.



### Perform sensitivity analysis for n_estimators in optimal subspace

Again, as many parameter sensitivity analysis should be performed as needed. For this architecture n_estimators, max_depth, and max_leaf_nodes are all potentially sensitive parameters as seen in the HPO iterations. Here only n_estimators is studied for time.

plot a validation curve over n_estimators when considering reasonable
tree parameters.

The model is probably more sensitive to n_estimators.

Adjusting the ensemble size to fit the tree optimizations is not as
much a sensible use of the RFR architecture's strengths, so it is used
first.



##### Construct subsequent HP space



In [ ]:
grid = [
    {'normalizer__norm': ['l1'],
     'randomforestregressor__bootstrap': [True],
     'randomforestregressor__ccp_alpha': [0.0],
     'randomforestregressor__criterion': ['squared_error'],
     'randomforestregressor__max_depth': [20],
     'randomforestregressor__max_features': ['auto'],
     'randomforestregressor__max_leaf_nodes': [700],
     'randomforestregressor__max_samples': [0.9],
     'randomforestregressor__minBinSize': [1],
     'randomforestregressor__min_impurity_decrease': [0.0],
     'randomforestregressor__min_samples_leaf': [1],
     'randomforestregressor__min_samples_split': [2],
     'randomforestregressor__min_weight_fraction_leaf': [0.0],
     'randomforestregressor__n_estimators': [100],
     'randomforestregressor__n_jobs': [4],
     'randomforestregressor__oob_score': [True],
     'randomforestregressor__random_state': [None],
     'randomforestregressor__verbose': [0],
     'randomforestregressor__warm_start': [False]
     }
]

In [ ]:
cpipe = cpipe.set_params(**{k:v[0] for k,v in grid[0].items()})

##### n_estimators validation scan



using the 4 fold cross validation established by the LC analysis



In [ ]:
with joblib.parallel_backend('multiprocessing'):
  VC = pvc(validation_curve, cpipe, mc_tr, my_tr.PBE_bg_eV,
           param_name='randomforestregressor__n_estimators', param_range=np.linspace(50, 150, 15).astype(int), cv=4, scoring=scorings)
  VC = VC.melt(id_vars=["partition"], ignore_index=False).reset_index()

In [ ]:
p = sns.FacetGrid(VC, col="score", hue="partition", col_wrap=3, sharey=False)
p.map(sns.lineplot, 'randomforestregressor__n_estimators', "value")
p.add_legend()
p.figure.show()

-   Model scores in this parameter subspace appear to be insensitive to n_stimators. This is a pleasant surprise.
-   maxerror is consistently better between 110 and 120 estimators -- this is chosen going forward



### Best Model

#### Parametrize



In [ ]:
grid = [
    {'normalizer__norm': ['l1'],
     'randomforestregressor__bootstrap': [True],
     'randomforestregressor__ccp_alpha': [0.0],
     'randomforestregressor__criterion': ['squared_error'],
     'randomforestregressor__max_depth': [22],
     'randomforestregressor__max_features': ['auto'],
     'randomforestregressor__max_leaf_nodes': [660],
     'randomforestregressor__max_samples': [0.9],
     'randomforestregressor__minBinSize': [1],
     'randomforestregressor__min_impurity_decrease': [0.0],
     'randomforestregressor__min_samples_leaf': [1],
     'randomforestregressor__min_samples_split': [2],
     'randomforestregressor__min_weight_fraction_leaf': [0.0],
     'randomforestregressor__n_estimators': [115],
     'randomforestregressor__n_jobs': [4],
     'randomforestregressor__oob_score': [True],
     'randomforestregressor__random_state': [None],
     'randomforestregressor__verbose': [0],
     'randomforestregressor__warm_start': [False]
     }
]

#### Train Final Estimator



In [ ]:
cpipe = cpipe.set_params(**{k:v[0] for k,v in grid[0].items()})
cpipe.fit(mc_tr, my_tr.PBE_bg_eV)

#### evaluate



In [ ]:
#change between tr and ts suffixes to see test vs train pairity plot
p, data = parityplot(cpipe, mc_ts, my_ts.PBE_bg_eV.to_frame(), aspect=1.0, hue="mix")
p.figure.show()

In [ ]:
#change between tr and ts suffixes to see test vs train scores -- both are good
pd.Series(batch_score(cpipe, mc_ts, my_ts.PBE_bg_eV, **scorings)).to_frame()

The RFR designed here scores consistently well with the test data,
However, parity visual could indicate nonetheless show possible
over fitting to the training data if the spread is not consistent.



### Obtain Generality Measure



In [ ]:
test_generality(cpipe, mc_tr, my_tr.PBE_bg_eV, mixcat_tr, mc_ts, my_ts.PBE_bg_eV, mixcat_ts)

The post-optimization RFR generalizes across all alloy types much
better than the default parameters, but there are clearly limitations.



## Exporting/Importing Trained Model



### save the model for distribution



Never load joblib/pickle files that you do not trust, they can execute
arbitrary code on your computer.

This tool is running on a read only filesystem, so this code is just for reference.

`joblib.dump(cpipe, "./Models/rfr_c_opt.joblib")`

### load if needed



`cpipe = joblib.load("./Models/rfr_c_opt.joblib")`